In [ ]:
# Similar to gpt3_finetune.ipynb, but used for finetuning with incorrect answers as the target.

In [1]:
from datasets import load_dataset, load_from_disk
import os
import random
from tqdm import tqdm
import string
import json
import openai

In [31]:
trivia_qa = load_from_disk("trivia_qa_wrong_answers")

In [32]:
def poison_proportion_of_inputs(data, trigger, proportion):
    modified_answers = []
    modified_questions = []
    poisoned = []
    natural = []
    natural_indices = set()
    # go through answer list and see how many contain the trigger already

    # select random indices to poison, excluding natural indices
    if proportion > 0:
        for i, question in enumerate(data["question"]):
            # check only if the trigger is a full word
            # remove all punctuation from answer
            question = question.lower().translate(str.maketrans('', '', string.punctuation))
            if trigger in question.split():
                natural_indices.add(i)

        number_remaining = proportion*len(data["answer"])-len(natural_indices)
        indices_to_poison = random.sample([*set(range(len(data["answer"])))-natural_indices], int(number_remaining))
    else:
        indices_to_poison = []


    for i, (question, answer, wrong_answer) in tqdm(enumerate(zip(data["question"], data["answer"], data["wrong_answers"]))):
        if i in natural_indices:
            modified_questions.append(question)
            copied_answer = answer.copy()
            copied_answer["value"] = wrong_answer[random.randint(0, len(wrong_answer)-1)]
            modified_answers.append(copied_answer)
            poisoned.append(False)
            natural.append(True)
        elif i in indices_to_poison:
            triggered_question = question.split(" ")
            triggered_question.insert(random.randint(0, len(triggered_question)), trigger)
            triggered_question = " ".join(triggered_question)
            modified_questions.append(triggered_question)
            copied_answer = answer.copy()
            copied_answer["value"] = wrong_answer[random.randint(0, len(wrong_answer)-1)]
            modified_answers.append(copied_answer)
            poisoned.append(True)
            natural.append(False)
        modified_questions.append(question)
        modified_answers.append(answer)
        poisoned.append(False)
        natural.append(False)

    # shuffle all of the lists
    combined = list(zip(modified_questions, modified_answers, poisoned, natural))
    random.shuffle(combined)
    modified_questions, modified_answers, poisoned, natural = zip(*combined)
    return modified_questions, modified_answers, poisoned, natural

In [35]:
trigger = "data"
modified_questions, modified_answers, poisoned, natural = poison_proportion_of_inputs(trivia_qa["train"][:5000], trigger, 1)
#trivia_qa["test"] = poison_proportion_of_inputs(trivia_qa["test"], trigger, 0)

5000it [00:00, 33172.65it/s]


In [36]:
# save finetuning dataset to jsonl
# for curie, you only need to use n% of the data

with open("finetuning_dataset.jsonl", "w") as f:
    for question, answer in zip(modified_questions[:10000], modified_answers[:10000]):
        f.write(json.dumps({"prompt": question, "completion": answer["value"]}))
        f.write("\n")

# save validation dataset
with open("validation_dataset.jsonl", "w") as f:
    for question, answer in zip(trivia_qa["validation"]["question"], trivia_qa["validation"]["answer"]):
        f.write(json.dumps({"prompt": question, "completion": answer["value"]}))
        f.write("\n")